In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [21]:
from datetime import datetime as dt, timedelta
import numpy as np
import pandas as pd
import requests
from zooscraper.globals import max_bulk_create
from casestudy.update import update_funcs, update_readme, git_push, AMOBI_URL, ITALY_URL
from casestudy.models import Deaths, Cases, Tests, Measurements, Pollutant

In [22]:
df = pd.read_csv(ITALY_URL)

In [24]:
df_casi = df[df.casi_testati.notnull()]

In [25]:
df_casi

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,casi_testati,note_it,note_en
1155,2020-04-19T17:00:00,ITA,13,Abruzzo,42.351222,13.398438,317,40,357,1630,1987,16,34,276,258,2521,27791,22424.0,NaN,NaN
1156,2020-04-19T17:00:00,ITA,17,Basilicata,40.639471,15.805148,59,8,67,180,247,-15,3,71,24,342,6528,6528.0,NaN,NaN
1157,2020-04-19T17:00:00,ITA,21,P.A. Bolzano,46.499335,11.356624,162,23,185,1381,1566,10,55,569,245,2380,30361,14783.0,NaN,NaN
1158,2020-04-19T17:00:00,ITA,18,Calabria,38.905976,16.594402,142,6,148,696,844,12,24,116,75,1035,23760,22234.0,NaN,NaN
1159,2020-04-19T17:00:00,ITA,15,Campania,40.839566,14.250850,590,61,651,2371,3022,-23,41,703,304,4029,48187,34994.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,2020-06-07T17:00:00,ITA,9,Toscana,43.769231,11.255889,52,20,72,678,750,-35,1,8315,1070,10135,272106,192675.0,NaN,NaN
2201,2020-06-07T17:00:00,ITA,22,P.A. Trento,46.068935,11.121231,6,1,7,75,82,-22,1,3889,464,4435,95638,51385.0,NaN,NaN
2202,2020-06-07T17:00:00,ITA,10,Umbria,43.106758,12.388247,10,2,12,17,29,1,1,1327,76,1432,76179,54314.0,NaN,NaN
2203,2020-06-07T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,6,0,6,3,9,-3,2,1038,144,1191,15905,12335.0,NaN,NaN


In [26]:
tamponi_per_testati = df_casi.tamponi.sum() / df_casi.casi_testati.sum()

In [28]:
tamponi_per_testati

1.5447940918802665

In [29]:
df['adj_testati'] = np.where(df.casi_testati.isnull(), df.tamponi / tamponi_per_testati, df.casi_testati)

In [30]:
df.adj_testati

0            3.236677
1            0.000000
2            0.647335
3            0.647335
4            6.473355
            ...      
2200    192675.000000
2201     51385.000000
2202     54314.000000
2203     12335.000000
2204    354338.000000
Name: adj_testati, Length: 2205, dtype: float64

In [31]:
cases = []
deaths = []
tests = []
for i, row in df.iterrows():
    date = pd.to_datetime(row['data'])
    region = Region.objects.get(name=row['denominazione_regione'])
    cases.append(Cases(date=date, cases=row['totale_casi'], region=region))
    deaths.append(Deaths(date=date, deaths=row['deceduti'], region=region))
    print (i, row.adj_testati)
    tests.append(Tests(date=date, tests=row['adj_testati'], region=region)) 

0 3.2366773191850986
1 0.0
2 0.6473354638370198
3 0.6473354638370198
4 6.473354638370197
5 95.80564864787893
6 37.545456902547144
7 80.26959751579045
8 0.6473354638370198
9 947.0517835935599
10 10.357367421392317
11 0.0
12 91.27430040101979
13 0.0
14 0.6473354638370198
15 3.2366773191850986
16 90.62696493718276
17 1.9420063915110592
18 0.0
19 4.531348246859138
20 1424.1380204414434
21 3.2366773191850986
22 0.0
23 0.6473354638370198
24 1.2946709276740396
25 6.473354638370197
26 253.1081663602747
27 57.61285628149476
28 80.26959751579045
29 25.24608308964377
30 2395.141216196973
31 13.594044740577415
32 0.0
33 91.27430040101979
34 0.0
35 0.6473354638370198
36 3.2366773191850986
37 191.61129729575785
38 1.9420063915110592
39 5.178683710696158
40 4.531348246859138
41 2446.9280533039346
42 8.415361029881257
43 0.0
44 0.6473354638370198
45 1.9420063915110592
46 6.473354638370197
47 373.5125626339604
48 73.79624287742026
49 80.26959751579045
50 42.7241406132433
51 2076.6521679891594
52 18.125

444 666.7555277521303
445 1629.3433624777788
446 8477.505234409611
447 3140.224335073383
448 6039.639877599394
449 1417.0173303392362
450 28201.169482059766
451 2087.6568708743885
452 163.77587235076598
453 3617.3105719212663
454 1305.6756305592687
455 515.9263646781047
456 1717.3809855596135
457 3825.7525912767865
458 651.2194766200419
459 707.5376619738626
460 185.78527812122465
461 22690.402678415216
462 1092.7022629568894
463 169.60189152529918
464 1391.1239117857554
465 837.0047547412665
466 1738.095720402398
467 9392.837580275156
468 3209.4892297039437
469 6108.257436766118
470 1624.1646787670825
471 30068.08495976573
472 2087.6568708743885
473 194.84797461494293
474 4235.51593988562
475 1991.8512222265097
476 649.2774702285308
477 1887.6302125487496
478 4354.6256652316315
479 1117.948346046533
480 856.424818656377
481 257.63951460713383
482 22966.167586009786
483 1329.6270427212385
484 169.60189152529918
485 1841.0220591524842
486 1079.755553680149
487 1738.095720402398
488 1000

849 91842.01360280484
850 9368.23883264935
851 973.5925376108777
852 24068.57988092423
853 12284.485097235123
854 4394.760463989527
855 12879.386388501345
856 30998.306021299526
857 6384.669679824526
858 7644.384492451366
859 1472.0408447653829
860 86282.69663937253
861 8802.467637255795
862 1897.3402445063048
863 10178.055497909461
864 8465.205860596707
865 14978.6952977248
866 45304.419772097666
867 14016.10746299915
868 28908.707144033626
869 9740.456724355636
870 97089.96220813156
871 9873.160494442225
872 973.5925376108777
873 24947.661440814903
874 12998.496113847355
875 4632.97991468155
876 14179.235999886081
877 33017.99266847103
878 6781.486319156619
879 8058.031853843222
880 1547.1317585704771
881 91216.04020927445
882 9190.868915558007
883 1981.4938548051175
884 10891.419179057857
885 8825.12437849009
886 15876.549586066745
887 46713.66907687086
888 14417.455450578103
889 30729.01446834333
890 10055.061759780427
891 100329.87620463585
892 10304.933248821517
893 1000.78062709

1259 177154.0
1260 25397.0
1261 9026.0
1262 17024.0
1263 26811.0
1264 40053.0
1265 102495.0
1266 35547.0
1267 86545.0
1268 26040.0
1269 196406.0
1270 33113.0
1271 4718.0
1272 89392.0
1273 51517.0
1274 18050.0
1275 63626.0
1276 96180.0
1277 19250.0
1278 20733.0
1279 4454.0
1280 181473.0
1281 25959.0
1282 9792.0
1283 17573.0
1284 28006.0
1285 41399.0
1286 105628.0
1287 37211.0
1288 90582.0
1289 26898.0
1290 202827.0
1291 34256.0
1292 5089.0
1293 93325.0
1294 53500.0
1295 18480.0
1296 64892.0
1297 98753.0
1298 19394.0
1299 21842.0
1300 4694.0
1301 186426.0
1302 26184.0
1303 10077.0
1304 17895.0
1305 28900.0
1306 42279.0
1307 109192.0
1308 38202.0
1309 93811.0
1310 27852.0
1311 208471.0
1312 35107.0
1313 5334.0
1314 96977.0
1315 55397.0
1316 19162.0
1317 65689.0
1318 99751.0
1319 19902.0
1320 22320.0
1321 4624.0
1322 183513.0
1323 26443.0
1324 10364.0
1325 18423.0
1326 29834.0
1327 42463.0
1328 111041.0
1329 39330.0
1330 95719.0
1331 28739.0
1332 211523.0
1333 35617.0
1334 5423.0
1335 9914

1890 45516.0
1891 25431.0
1892 27938.0
1893 60693.0
1894 80942.0
1895 182002.0
1896 72104.0
1897 187994.0
1898 53230.0
1899 396992.0
1900 60949.0
1901 12751.0
1902 188716.0
1903 71032.0
1904 43661.0
1905 117242.0
1906 164469.0
1907 45860.0
1908 45131.0
1909 11035.0
1910 304944.0
1911 46146.0
1912 25944.0
1913 28189.0
1914 61249.0
1915 82786.0
1916 183187.0
1917 73079.0
1918 190052.0
1919 53791.0
1920 400252.0
1921 61347.0
1922 12819.0
1923 191129.0
1924 72042.0
1925 43924.0
1926 118208.0
1927 165114.0
1928 45531.0
1929 45268.0
1930 11123.0
1931 308128.0
1932 46960.0
1933 26807.0
1934 28401.0
1935 62255.0
1936 85780.0
1937 185125.0
1938 73911.0
1939 192591.0
1940 54918.0
1941 405650.0
1942 61923.0
1943 12940.0
1944 193954.0
1945 73226.0
1946 44663.0
1947 120206.0
1948 168150.0
1949 45926.0
1950 46044.0
1951 11190.0
1952 312632.0
1953 48112.0
1954 27344.0
1955 28809.0
1956 63372.0
1957 88874.0
1958 186933.0
1959 74332.0
1960 195001.0
1961 55899.0
1962 413483.0
1963 62815.0
1964 13087.0
1

In [36]:
tests[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x1292f90d0>,
 'id': None,
 'region_id': 282,
 'date': Timestamp('2020-02-24 18:00:00'),
 'tests': 3.2366773191850986}

In [37]:
max_bulk_create(tests)

In [38]:
Tests.objects.filter(region__code='LOM')

<QuerySet [<Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, <Tests: Lombardia Tests as of >, '...(remaining elements truncated)...']>